In [1]:
!pip install python-pptx

   ---------------------------------------- 0.0/472.8 kB ? eta -:--:--
    --------------------------------------- 10.2/472.8 kB ? eta -:--:--
   --- ----------------------------------- 41.0/472.8 kB 495.5 kB/s eta 0:00:01
   ------------------- -------------------- 235.5/472.8 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 472.8/472.8 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/3.8 MB 9.4 MB/s eta 0:00:01
   ------------ --------------------------- 1.2/3.8 MB 12.4 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 12.5 MB/s eta 0:00:01
   -------------------- ------------------- 2.0/3.8 MB 11.4 MB/s eta 0:00:01
   -------------------------- ------------- 2.5/3.8 MB 11.4 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/3.8 MB 10.2 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.8 MB 10.0 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

from pptx import Presentation
from pptx.util import Inches, Pt
import textwrap

def get_bible_text(bible_json:dict, bible_head_dict:dict, head_str:str, f_num:str, s_start_num:str, s_end_num:str = '0') -> list:
    # 성경 리스트로 불러오기
    the_text_li = []
    the_text_li.append(s_start_num+'. '+bible_json[head_str][f_num][s_start_num])
    if s_end_num == '0':
        return the_text_li
    else:
        for idx in range(int(s_start_num)+1, int(s_end_num)+1):
            the_text_li.append(str(idx)+'. '+bible_json[head_str][f_num][str(idx)])

    # ppt 파일 이름 생성
    ppt_file_nm = bible_head_dict[head_str]+f_num+'_'+s_start_num
    if s_end_num!= '0':
        ppt_file_nm = ppt_file_nm +'-'+s_end_num
    
    return the_text_li, ppt_file_nm

# the_text_li, ppt_file_nm = get_bible_text(bible_json, '창', '1', '1', '4')

def add_text_to_textbox(text_box_tf_p, text, text_box_size, font_size):
    """텍스트 상자에 텍스트를 추가하고 줄바꿈을 처리합니다.

    Args:
        textbox: 텍스트 상자 객체.
        text: 추가할 텍스트 문자열.
        width_ratio: 텍스트 상자 너비 대비 문자 수 비율 (조절 필요).
    """
    wrapped_text = textwrap.wrap(text, width=int(Inches(text_box_size) / (Pt(font_size)*0.8)))
    text_box_tf_p.text = "\n".join(wrapped_text)

def get_bible_ppt(image_files_path, the_text_li, ppt_file_nm):
    # 이미지와 텍스트리스트, 파일명을 받아 ppt로 구성해서 저장하는 함수

    # 프레젠테이션 객체 생성
    prs = Presentation()

    # 각 이미지에 대해 슬라이드 추가
    for idx in range(len(the_text_li)):
        the_text = the_text_li[idx]
        prs.slide_width = Inches(16) 
        prs.slide_height = Inches(9)
        slide_layout = prs.slide_layouts[1]  # 빈 슬라이드 레이아웃 선택
        slide = prs.slides.add_slide(slide_layout)

        # 이미지 추가
        left = Inches(0)
        top = Inches(0)
        height = Inches(9)
        
        slide.shapes.add_picture(image_files_path, left, top, width=Inches(16), height=Inches(9))
        # slide.shapes.add_picture(image_files, left, top, height=height)

        # 제목
        shape = slide.shapes.title
        shape.text = ':'.join(ppt_file_nm.split('_'))


        #title_box = slide.shapes.add_textbox(left=Inches(1), top=Inches(0.2), width=Inches(14), height=Inches(6))
        #title_box_tf = title_box.text_frame
        #title_box_tf_p = title_box_tf.add_paragraph()
        #title_box_tf_p.text = ':'.join(ppt_file_nm.split('_'))
        #title_box_tf_p.font.size = Pt(45)
        #title_box_tf_p.font.bold = True
        #try:
        #    title_box_tf_p.font.name = '맑은 고딕'
        #except:
        #    pass

        # 내용
        text_box = slide.shapes.add_textbox(left=Inches(1), top=Inches(1.2), width=Inches(14), height=Inches(6))
        text_box_tf = text_box.text_frame
        text_box_tf_p = text_box_tf.add_paragraph()
        add_text_to_textbox(text_box_tf_p, the_text, text_box_size=13, font_size=36)
        text_box_tf_p.font.size = Pt(40)
        try:
            text_box_tf_p.font.name = '넥슨 풋볼고딕 B'
        except:
            pass
        
        #tf = body_shape.text_frame
        #tf.text = the_text

    # 파일 저장
    
    prs.save('./ppts/'+ppt_file_nm+'.pptx')

In [13]:
head_str = '요' 
f_num  = '1'
s_start_num = '1'
s_end_num = '10'

In [15]:
import json

with open('./data/index_bible2.json', 'r', encoding='utf8') as b_json:
    bible_json = json.load(b_json)

with open('./data/bible_head_dict.json', 'r', encoding='utf8') as b_json:
    bible_head_dict = json.load(b_json)
    
image_files_path = './data/ppt배경.png'  # 여기에 이미지 파일 경로를 추가하세요

the_text_li, ppt_file_nm = get_bible_text(bible_json, bible_head_dict, head_str, f_num, s_start_num, s_end_num)
get_bible_ppt(image_files_path, the_text_li, ppt_file_nm)

In [297]:
bible_head_dict = {'창':'창세기', '출':'출애굽기', '레':'레위기', '민':'민수기', '신':'신명기', '수':'여호수아', '삿':'사사기', '룻':'룻기', '삼상':'사무엘상', '삼하':'사무엘하', '왕상':'열왕기상', '왕하':'열왕기하', '대상':'역대상', '대하':'역대하', '스':'에스라', '느':'느헤미야', '에':'에스더', '욥':'욥기', '시':'시편', '잠':'잠언', '전':'전도서', '아':'아가', '사':'이사야', '렘':'예레미야', '애':'예레미야애가', '겔':'에스겔', '단':'다니엘', '호':'호세아', '욜':'요엘', '암':'아모스', '옵':'오바댜', '욘':'요나', '미':'미가', '나':'나훔', '합':'하박국', '습':'스바냐', '학':'학개', '슥':'스가랴', '말':'말라기', '마':'마태복음', '막':'마가복음', '눅':'누가복음', '요':'요한복음', '행':'사도행전', '롬':'로마서', '고전':'고린도전서', '고후':'고린도후서', '갈':'갈라디아서', '엡':'에베소서', '빌':'빌립보서', '골':'골로새서', '살전':'데살로니가전서', '살후':'데살로니가후서', '딤전':'디모데전서', '딤후':'디모데후서', '딛':'디도서', '몬':'빌레몬서', '히':'히브리서', '약':'야고보서', '벧전':'베드로전서', '벧후':'베드로후서', '요일':'요한일서', '요이':'요한이서', '요삼':'요한삼서', '유':'유다서', '계':'요한계시록'}

In [298]:
with open('./data/bible_head_dict.json', 'w', encoding='utf8') as f:
    json.dump(bible_head_dict, f)

In [162]:
ppt_file_nm

'요1_1-10'

In [ ]:
# bible.json 전처리해서 저장 코드

# import re
# from collections import defaultdict

# new_bible_dict = defaultdict(str)
# error_dict = dict()
# for raw_key in bible_json.keys():
#     try:
#         head_str = re.sub(r'[0-9]+:[0-9]+', '', raw_key).strip()
#         f_num_idx_li = re.search(r'[0-9]+:[0-9]+', raw_key).span()
#         f_num, s_num = raw_key[f_num_idx_li[0]:f_num_idx_li[1]].split(':')
#     except:
#         error_dict[raw_key] = bible_json[raw_key].strip()
#     if len(new_bible_dict[head_str]) == 0:
#         new_bible_dict[head_str] = defaultdict(str)
#     if len(new_bible_dict[head_str][f_num]) == 0:
#         new_bible_dict[head_str][f_num] = defaultdict(str)
    
#     new_bible_dict[head_str][f_num][s_num] = bible_json[raw_key].strip()

In [ ]:
#with open('./fewfewf', 'w', encoding='utf8') as f:
#    json.dump(bible_json, f)

In [ ]:
# def get_bible_ppt(image_files_path, the_text_li, ppt_file_nm):
#     # 이미지와 텍스트리스트, 파일명을 받아 ppt로 구성해서 저장하는 함수

#     # 프레젠테이션 객체 생성
#     prs = Presentation()

#     # 각 이미지에 대해 슬라이드 추가
#     for idx in range(len(the_text_li)):
#         the_text = the_text_li[idx]
#         prs.slide_width = Inches(16) 
#         prs.slide_height = Inches(9)
#         slide_layout = prs.slide_layouts[1]  # 빈 슬라이드 레이아웃 선택
#         slide = prs.slides.add_slide(slide_layout)

#         # 이미지 추가
#         left = Inches(0)
#         top = Inches(0)
#         height = Inches(9)
        
#         slide.shapes.add_picture(image_files_path, left, top, width=Inches(16), height=Inches(9))
#         # slide.shapes.add_picture(image_files, left, top, height=height)

#         # 제목
#         title_box = slide.shapes.add_textbox(left=Inches(1), top=Inches(0.2), width=Inches(14), height=Inches(6))
#         title_box_tf = title_box.text_frame
#         title_box_tf_p = title_box_tf.add_paragraph()
#         title_box_tf_p.text = ':'.join(ppt_file_nm.split('_'))
#         title_box_tf_p.font.size = Pt(45)
#         title_box_tf_p.font.bold = True
#         try:
#             title_box_tf_p.font.name = '맑은 고딕'
#         except:
#             pass

#         # 내용
#         text_box = slide.shapes.add_textbox(left=Inches(1), top=Inches(1.2), width=Inches(14), height=Inches(6))
#         text_box_tf = text_box.text_frame
#         text_box_tf_p = text_box_tf.add_paragraph()
#         add_text_to_textbox(text_box_tf_p, the_text, text_box_size=13, font_size=36)
#         text_box_tf_p.font.size = Pt(40)
#         try:
#             text_box_tf_p.font.name = '넥슨 풋볼고딕 B'
#         except:
#             pass
        
#         #tf = body_shape.text_frame
#         #tf.text = the_text

#     # 파일 저장
    
#     prs.save('./ppts/'+ppt_file_nm+'.pptx')

In [ ]:
wrapped_text = textwrap.wrap('new_bible_dict[head_str][f_num][s_num] = bible_json[raw_key].strip()', 
                             width=int(Inches(13) / Pt(40)))
print(wrapped_text)
"\n".join(wrapped_text)